# 🏛️ Entrenamiento de Traductor Latín→Español - Fase 1

## Objetivo
Entrenar un modelo mT5-small para traducir latín clásico a español.

**Fase 1**: Modelo base con ~30,000 pares (BLEU esperado: ~30-35)

---

## ⚙️ Configuración Inicial

In [ ]:
# ============================================
# SECCIÓN 1: CONEXIÓN A GOOGLE DRIVE
# ============================================
# ¿Por qué? Para guardar checkpoints y no perder progreso

from google.colab import drive
drive.mount('/content/drive')

# Crear directorio de trabajo
!mkdir -p /content/drive/MyDrive/latin_translator_phase1
%cd /content/drive/MyDrive/latin_translator_phase1

print("✅ Google Drive conectado")
print("📁 Directorio: /content/drive/MyDrive/latin_translator_phase1")

In [ ]:
# ============================================
# SECCIÓN 2: INSTALACIÓN DE DEPENDENCIAS
# ============================================

!pip install -q transformers datasets sentencepiece sacrebleu accelerate

print("✅ Dependencias instaladas")

In [ ]:
# ============================================
# SECCIÓN 3: VERIFICACIÓN DE GPU
# ============================================

import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU disponible: {gpu_name}")
    print(f"💾 Memoria GPU: {gpu_memory:.1f} GB")
else:
    print("❌ GPU no disponible")
    print("⚠️ Ve a: Runtime > Change runtime type > GPU (T4)")

---

## 📊 Preparación de Datos

Vamos a usar múltiples fuentes para el corpus de Fase 1.

In [ ]:
# ============================================
# SECCIÓN 4: DESCARGA DE CORPUS
# ============================================
# Usaremos el corpus OPUS que tiene pares latín-español

import json
import requests
from pathlib import Path

# Crear directorio de datos
!mkdir -p data

# Opción 1: Subir tus propios datos
print("📤 OPCIÓN 1: Sube tus archivos train.json y validation.json")
print("   (Si ya los tienes preparados localmente)")
print()
print("📥 OPCIÓN 2: Usar datos de ejemplo")
print("   (Continuaremos con datos mínimos para probar)")

# Datos de ejemplo (classical samples)
example_data = [
    {"latin": "Gallia est omnis divisa in partes tres.", 
     "spanish": "Toda la Galia está dividida en tres partes."},
    {"latin": "Veni, vidi, vici.", 
     "spanish": "Vine, vi, vencí."},
    {"latin": "Alea iacta est.", 
     "spanish": "La suerte está echada."},
    {"latin": "Carpe diem.", 
     "spanish": "Aprovecha el día."},
    {"latin": "Errare humanum est.", 
     "spanish": "Errar es humano."},
]

# Guardar datos de ejemplo
with open('data/example_train.json', 'w', encoding='utf-8') as f:
    json.dump(example_data * 100, f, ensure_ascii=False, indent=2)  # Repetir para tener más datos

with open('data/example_validation.json', 'w', encoding='utf-8') as f:
    json.dump(example_data[:2], f, ensure_ascii=False, indent=2)

print("✅ Datos de ejemplo creados")
print("⚠️ IMPORTANTE: Para entrenamiento real, necesitas subir corpus más grande")

In [ ]:
# ============================================
# SECCIÓN 5: CARGAR DATOS
# ============================================

from datasets import Dataset, DatasetDict

def load_data(train_path='data/train.json', val_path='data/validation.json'):
    """
    Carga datos de entrenamiento y validación.
    Si no existen, usa los de ejemplo.
    """
    # Intentar cargar datos reales
    try:
        with open(train_path, 'r', encoding='utf-8') as f:
            train_data = json.load(f)
        with open(val_path, 'r', encoding='utf-8') as f:
            val_data = json.load(f)
        print("✅ Datos personalizados cargados")
    except FileNotFoundError:
        print("⚠️ Usando datos de ejemplo")
        with open('data/example_train.json', 'r', encoding='utf-8') as f:
            train_data = json.load(f)
        with open('data/example_validation.json', 'r', encoding='utf-8') as f:
            val_data = json.load(f)
    
    # Convertir a Dataset
    train_dataset = Dataset.from_dict({
        'latin': [item['latin'] for item in train_data],
        'spanish': [item['spanish'] for item in train_data]
    })
    
    val_dataset = Dataset.from_dict({
        'latin': [item['latin'] for item in val_data],
        'spanish': [item['spanish'] for item in val_data]
    })
    
    return DatasetDict({
        'train': train_dataset,
        'validation': val_dataset
    })

# Cargar datos
dataset = load_data()

print(f"\n📊 Estadísticas:")
print(f"   - Entrenamiento: {len(dataset['train'])} pares")
print(f"   - Validación: {len(dataset['validation'])} pares")
print(f"\n📝 Ejemplo:")
print(f"   Latin: {dataset['train'][0]['latin']}")
print(f"   Spanish: {dataset['train'][0]['spanish']}")

---

## 🤖 Configuración del Modelo

In [ ]:
# ============================================
# SECCIÓN 6: CARGAR MODELO mT5-small
# ============================================

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "google/mt5-small"

print(f"📥 Cargando modelo: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

print(f"✅ Modelo cargado")
print(f"📊 Parámetros: {model.num_parameters():,}")
print(f"💾 Tamaño aproximado: ~300MB")

In [ ]:
# ============================================
# SECCIÓN 7: PREPROCESAMIENTO
# ============================================

def preprocess_function(examples):
    """
    Preprocesa los datos para mT5.
    Añade prefijo de tarea y tokeniza.
    """
    # Prefijo de tarea
    inputs = ["translate Latin to Spanish: " + text for text in examples['latin']]
    targets = examples['spanish']
    
    # Tokenizar
    model_inputs = tokenizer(
        inputs,
        max_length=128,
        truncation=True,
        padding='max_length'
    )
    
    labels = tokenizer(
        targets,
        max_length=128,
        truncation=True,
        padding='max_length'
    )
    
    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs

# Aplicar preprocesamiento
print("🔄 Preprocesando datos...")
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)

print("✅ Datos preprocesados")

---

## 🎯 Entrenamiento

In [ ]:
# ============================================
# SECCIÓN 8: CONFIGURACIÓN DE ENTRENAMIENTO
# ============================================

from transformers import TrainingArguments, Trainer
import numpy as np

training_args = TrainingArguments(
    # Directorio de salida (en Google Drive)
    output_dir="./checkpoints",
    
    # Guardado de checkpoints
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,  # Mantener solo últimos 3
    
    # Evaluación
    evaluation_strategy="steps",
    eval_steps=500,
    
    # Hiperparámetros
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    
    # Optimizaciones
    fp16=True,  # Precisión mixta
    gradient_accumulation_steps=2,
    
    # Logging
    logging_dir="./logs",
    logging_steps=100,
    
    # Otros
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none"
)

print("✅ Configuración de entrenamiento lista")
print(f"\n📊 Parámetros:")
print(f"   - Épocas: {training_args.num_train_epochs}")
print(f"   - Learning rate: {training_args.learning_rate}")
print(f"   - Batch size: {training_args.per_device_train_batch_size}")
print(f"   - Checkpoints cada: {training_args.save_steps} pasos")

In [ ]:
# ============================================
# SECCIÓN 9: MÉTRICAS
# ============================================

from datasets import load_metric

metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    """
    Calcula BLEU score.
    """
    preds, labels = eval_preds
    
    # Decodificar
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # BLEU
    result = metric.compute(
        predictions=decoded_preds,
        references=[[label] for label in decoded_labels]
    )
    
    return {"bleu": result["score"]}

print("✅ Métricas configuradas")

In [ ]:
# ============================================
# SECCIÓN 10: INICIAR ENTRENAMIENTO
# ============================================

import os

# Verificar checkpoints existentes
checkpoints = [d for d in os.listdir("./checkpoints") if d.startswith("checkpoint-")] if os.path.exists("./checkpoints") else []

if checkpoints:
    latest = sorted(checkpoints, key=lambda x: int(x.split("-")[1]))[-1]
    checkpoint_path = f"./checkpoints/{latest}"
    print(f"🔄 Reanudando desde: {checkpoint_path}")
    resume_from = checkpoint_path
else:
    print("🆕 Iniciando entrenamiento desde cero")
    resume_from = None

# Crear Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("\n" + "="*60)
print("🚀 INICIANDO ENTRENAMIENTO")
print("="*60)
print("\n⏱️ Tiempo estimado: 8-12 horas (para corpus completo)")
print("💾 Checkpoints se guardan cada 500 pasos en Google Drive")
print("🔄 Puedes cerrar esta pestaña - el progreso se guarda")
print("\n" + "="*60 + "\n")

# ENTRENAR
trainer.train(resume_from_checkpoint=resume_from)

print("\n" + "="*60)
print("✅ ENTRENAMIENTO COMPLETADO")
print("="*60)

In [ ]:
# ============================================
# SECCIÓN 11: GUARDAR MODELO FINAL
# ============================================

trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

print("✅ Modelo final guardado en: ./final_model")
print("\n📥 Para descargar:")
print("   1. Ve a Google Drive")
print("   2. Navega a: MyDrive/latin_translator_phase1/final_model")
print("   3. Descarga la carpeta completa")
print("   4. Colócala en tu proyecto: models/latin_translator_v1.0")

---

## 🧪 Pruebas del Modelo

In [ ]:
# ============================================
# SECCIÓN 12: PROBAR TRADUCCIONES
# ============================================

def translate(latin_text, model_path="./final_model"):
    """
    Traduce texto latino a español.
    """
    input_text = f"translate Latin to Spanish: {latin_text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
    
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

# Probar con ejemplos
test_sentences = [
    "Gallia est omnis divisa in partes tres.",
    "Veni, vidi, vici.",
    "Alea iacta est.",
    "Carpe diem."
]

print("🧪 Probando traducciones:\n")
for latin in test_sentences:
    spanish = translate(latin)
    print(f"Latin:   {latin}")
    print(f"Spanish: {spanish}")
    print()

---

## 📊 Evaluación Final

In [ ]:
# ============================================
# SECCIÓN 13: EVALUACIÓN FINAL
# ============================================

# Evaluar en conjunto de validación
eval_results = trainer.evaluate()

print("📊 Resultados Finales:")
print(f"   - Loss: {eval_results['eval_loss']:.4f}")
print(f"   - BLEU: {eval_results['eval_bleu']:.2f}")
print()
print("🎯 Interpretación:")
if eval_results['eval_bleu'] > 40:
    print("   ✅ Excelente calidad")
elif eval_results['eval_bleu'] > 30:
    print("   ✅ Buena calidad")
elif eval_results['eval_bleu'] > 20:
    print("   ⚠️ Calidad aceptable - considera Fase 2")
else:
    print("   ❌ Calidad baja - necesitas más datos")

---

## 🎉 ¡Entrenamiento Completado!

### Próximos Pasos:

1. **Descargar modelo**: Descarga la carpeta `final_model` de Google Drive
2. **Integrar en aplicación**: Coloca el modelo en tu proyecto local
3. **Evaluar calidad**: Prueba con textos reales
4. **Fase 2** (opcional): Si necesitas mejor calidad, continúa entrenamiento con más datos

### Información del Modelo:

- **Nombre**: `latin_translator_v1.0`
- **Arquitectura**: mT5-small
- **Tamaño**: ~300MB
- **Datos de entrenamiento**: Fase 1
- **BLEU score**: (ver arriba)

---

**¿Preguntas?** Revisa la guía en `docs/ai_training_guide.md`